In [ ]:
# | default_exp routes.pdp

In [ ]:
# | export
import httpx
import pandas as pd
import io

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [ ]:
# | export
class PDP_NotRetrieved(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        pdp_id=None,
    ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=pdp_id,
            function_name=function_name,
            status=status,
            message=message,
        )

In [ ]:
# | export
async def get_pdp_policies(
    auth: dmda.DomoAuth,
    dataset_id: str,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"http://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth, 
        url=url, 
        method="GET", 
        debug_api=debug_api, 
        # headers= {'accept': 'application/json'},
        # params = {'options':'load_associations,load_filters,include_open_policy'},
        is_follow_redirects=True
    )

    if len(res.response) == 0 or not res.is_success:
        raise PDP_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name="get_pdp_policies",
            status=res.status,
            message="failed to retrieve pdp policies",
        )

    return res

### Sample implementation for get_pdp_policies

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

try:
    res = await get_pdp_policies(auth=token_auth, dataset_id=dataset_id)
    print(res)
except PDP_NotRetrieved as e:
    print(e)


ResponseGetData(status=200, response=[{'name': 'Test 456', 'filterGroupId': 1156, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'type': 'user', 'userIds': [2072616249], 'dataSourcePermissions': False, 'resources': {'USER': ['2072616249']}, 'parameters': [{'name': 'Employee ID', 'value': '465', 'values': ['465'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': False}, {'name': 'Country', 'value': 'Philippines', 'values': ['Philippines'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': False}]}, {'name': 'Test 94', 'filterGroupId': 1157, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'type': 'user', 'userIds': [1893952720], 'groupIds': [1259653287], 'dataSourcePermissions': False, 'resources': {'GROUP': ['1259653287'], 'USER': ['1893952720']}, 'parameters': [{'name': 'Employee ID', 'value': '465', 'values': ['465'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': False}, {'name': 'Customer ID', 'value': '8978'

In [ ]:

# class SearchPDP_Error(de.DomoError):
#     def __init__(self, status, message, domo_instance, function_name = "search_pdp_by_name"):
#         super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)
        
# def search_pdp_policies_by_name(
#         # used to return pdp policy info, search by name
#         search_name: str,
#         result_list: list[dict], # this is the res.response from get_pdp_policies -- should be list of dict
#         is_exact_match: bool = True
#         ):
    
#     if is_exact_match:
#         policy_search = next((policy for policy in result_list if policy.name == search_name), None)
#         #print(policy_search)   
         
#         if not policy_search:
#             raise SearchPDP_Error(
#                 status='',
#                 message=f'There is no policy named "{search_name}"',
#                 domo_instance=''
#             )  
          
#         return policy_search
#     else:
#         policy_search = [policy for policy in result_list if search_name.lower() in policy.name.lower()]
#         if not policy_search:
#             raise SearchPDP_Error(
#                 status='',
#                 message=f'There is no policy name containing "{search_name}"',
#                 domo_instance=''
#             )  
        
#         return policy_search

In [ ]:
# pdp_policy_search = search_pdp_policies_by_name(search_name='1', result_list=res.response, is_exact_match=False)
# pdp_policy_search

In [ ]:

# class SearchPDP_Error(de.DomoError):
#     def __init__(self, status, message, domo_instance, function_name = "search_pdp_by_name"):
#         super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)

# async def search_pdp_policies_by_id(
#         # used to return pdp policy info, search by id
#         search_id: str,
#         result_list: list[dict], # this is the res.response from get_pdp_policies -- should be list of dict
#         ):
    
#     policy_search = next((policy for policy in result_list if policy.filter_group_id == search_id), None)
#     #print(policy_search)   
         
#     if not policy_search:
#         raise SearchPDP_Error(
#             status='',
#             message=f'There is no policy id "{search_id}"',
#             domo_instance=''
#         )  
          
#     return policy_search


## generate body for create_policy endpoint


In [ ]:
# | export
def generate_policy_parameter_simple(
    column_name: str,
    type: str = "COLUMN",
    column_values_ls: list[str] = None,
    operator="EQUALS",
    ignore_case: bool = True,
):
    if not isinstance(column_values_ls, list):
        column_values_ls = [column_values_ls]

    return {
        "type": type,
        "name": column_name,
        "values": column_values_ls,
        "operator": operator,
        "ignoreCase": ignore_case,
    }

In [ ]:
# | export
def generate_policy_body(
    policy_name: str,
    dataset_id: str,
    parameters_ls: list[dict],  # generated by generate_policy_parameter_simple method
    policy_id: str = None,  # only included if updating existing policy
    user_ids: list[str] = None,
    group_ids: list[str] = None,
    virtual_user_ids: list[str] = None,
):
    if not user_ids:
        user_ids = []

    if not group_ids:
        group_ids = []

    if not virtual_user_ids:
        virtual_user_ids = []

    if not isinstance(parameters_ls, list):
        parameters_ls = [parameters_ls]


    body = {
        "name": policy_name,
        "dataSourceId": dataset_id,
        "userIds": user_ids,
        "virtualUserIds": virtual_user_ids,
        "groupIds": group_ids,
        "dataSourcePermissions": False,
        "parameters": parameters_ls,
    }

    if policy_id:
        body.update({"filterGroupId": policy_id})

    return body

## Policy CRUD Operations

### Create PDP Policy

In [ ]:
# | export
class CreatePolicy_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name = "create_policy"):
        super().__init__(function_name = function_name, status = status, message = message , domo_instance = domo_instance)

async def create_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    override_same_name: bool = False,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups"

    if debug_api:
        print(url)
    
    if override_same_name:
        print(f'Creating policy...')
        res = await gd.get_data(
            auth=auth,
            url=url,
            method="POST",
            body=body,
            debug_api=debug_api,
            session=session,
        )
    else:
        existing_policies = await get_pdp_policies(auth=auth, dataset_id=dataset_id) 
        
        if existing_policies.is_success:
            try:
                policy_exists = search_pdp_policies_by_name(search_name=body.get('name'), result_list=existing_policies.response, is_exact_match=True)
            except:
                policy_exists = False
            
            if policy_exists:
                raise CreatePolicy_Error(
                    status='',
                    message= f'Policy name already exists--avoid creating pdp policies with the same name..To override, rerun and set "override_same_name=True"',
                    domo_instance=auth.domo_instance,
                )
            else:
                print(f'Creating policy...')
                res = await gd.get_data(
                auth=auth,
                url=url,
                method="POST",
                body=body,
                debug_api=debug_api,
                session=session,
            )

    return res

#### sample implementation of create_policy, generate_policy_parameter_simple and generate_policy_body

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameter = generate_policy_parameter_simple(
    column_name="Employee ID",
    column_values_ls=['123'],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    policy_name="Test Policy Name Employee 1789",
    dataset_id = dataset_id,
    #policy_id="123",  # including the policy_id updates that existing policy
    parameters_ls=sample_parameter,
    user_ids=['1893952720']
)

res = await create_policy(
    auth = token_auth,
    dataset_id = dataset_id,
    body = body,
    override_same_name=False
)
res.response

Creating policy...


{'name': 'Test Policy Name Employee 1789',
 'filterGroupId': 1180,
 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3',
 'policySetId': 1130,
 'userIds': [1893952720],
 'dataSourcePermissions': False,
 'parameters': [{'name': 'Employee ID',
   'value': '123',
   'values': ['123'],
   'type': 'COLUMN',
   'operator': 'EQUALS',
   'not': False,
   'ignoreCase': True}]}

`<<<<<<< HEAD`

In [ ]:
### Update PDP Policy

`=======`

### Update PDP Policy

`>>>>>>> 8116b12ebbf79ae276997024c2e098dad68dc8ba`

In [ ]:
# | export
async def update_policy(
    auth: dmda.DomoAuth,
    dataset_id: str,
    policy_id: str,
    body: dict,  # generated using generate_policy_parameter_simple & generate_policy_body
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/{policy_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of update_policy

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

sample_parameters = generate_policy_parameter_simple(
    column_name="Employee ID",
    column_values_ls=["123"],
    operator="EQUALS",
    ignore_case=True,
)

body = generate_policy_body(
    "Test Policy Name Employee 123",
    dataset_id=dataset_id,
    # policy_id='1153',
    parameters_ls=[sample_parameters],
    user_ids=['1893952720']
)

res = await update_policy(
    auth=token_auth,
    dataset_id=dataset_id,
    policy_id='1178',
    body=body,
    debug_api=True
)

res

https://domo-community.domo.com/api/query/v1/data-control/0e7f7755-eccc-4612-890c-c3dc3d5002c3/filter-groups/1178
🐛 debugging get_data
{'body': {'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3',
          'dataSourcePermissions': False,
          'groupIds': [],
          'name': 'Test Policy Name Employee 123',
          'parameters': [{'ignoreCase': True,
                          'name': 'Employee ID',
                          'operator': 'EQUALS',
                          'type': 'COLUMN',
                          'values': ['123']}],
          'userIds': ['1893952720'],
          'virtualUserIds': []},
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '2f4cb212853732ef796abd46077cf1162c0a3ed7cf1ff079'},
 'method': 'PUT',
 'params': None,
 'url': 'https://domo-community.domo.com/api/query/v1/data-control/0e7f7755-eccc-4612-890c-c3dc3d5002c3/

ResponseGetData(status=200, response={'name': 'Test Policy Name Employee 123', 'filterGroupId': 1178, 'dataSourceId': '0e7f7755-eccc-4612-890c-c3dc3d5002c3', 'userIds': [1893952720], 'dataSourcePermissions': False, 'parameters': [{'name': 'Employee ID', 'value': '123', 'values': ['123'], 'type': 'COLUMN', 'operator': 'EQUALS', 'not': False, 'ignoreCase': True}]}, is_success=True)

### Delete PDP Policy

In [ ]:
# | export
async def delete_policy(
    auth: dmda.DomoAuth,
    dataset_id: str = None,
    policy_id: str = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}/filter-groups/{policy_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="DELETE",
        debug_api=debug_api,
        session=session,
    )

    return res

### sample implementation of delete_policy

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

res = await delete_policy(
    auth=token_auth, 
    dataset_id=dataset_id, 
    policy_id='1179',
    debug_api=True
)

https://domo-community.domo.com/api/query/v1/data-control/0e7f7755-eccc-4612-890c-c3dc3d5002c3/filter-groups/1179
🐛 debugging get_data
{'body': None,
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': '2f4cb212853732ef796abd46077cf1162c0a3ed7cf1ff079'},
 'method': 'DELETE',
 'params': None,
 'url': 'https://domo-community.domo.com/api/query/v1/data-control/0e7f7755-eccc-4612-890c-c3dc3d5002c3/filter-groups/1179'}
get_data: no body
get_data_response <Response [404 Not Found]>


### Enable / Disable PDP on dataset (toggle_pdp)

In [ ]:
# | export

async def toggle_pdp(
        auth: dmda.DomoAuth,
        dataset_id: str,
        is_enable: bool = True,
        debug_api: bool = False,
        session: httpx.AsyncClient = None
    )-> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/query/v1/data-control/{dataset_id}"

    if debug_api:
        print(url)

    body = {"enabled": is_enable,
            "external": False  # not sure what this parameter does
            }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res


#### sample implementation of toggle_pdp

In [ ]:
import os
from pprint import pprint

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

dataset_id = os.environ["DOJO_DATASET_ID"]

res = await toggle_pdp(
    auth=token_auth,
    dataset_id=dataset_id,
    is_enable = True,
)
res

ResponseGetData(status=200, response={'enabled': True, 'secured': False, 'external': False}, is_success=True)

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()